# DATA CLEANING

In [ ]:
import  pandas as pd 
df = pd.read_csv("shopping_behavior.csv")
df

In [ ]:
df['Review Rating']=df['Review Rating'].round().astype(int)
df

In [ ]:
df.describe()

In [ ]:
df.info()


In [ ]:
category_of_columns=['Gender','Item Purchased','Category','Location','Size','Color','Season','Subscription Status','Shipping Type','Discount Applied','Promo Code Used','Payment Method','Frequency of Purchases']
for i in category_of_columns:
    print(f"\n Unique values and count for {i}:/n{df[i].value_counts()}")

In [ ]:
mv=df.isnull().sum()
mv

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
category_count=df['Category'].value_counts()
category_count

# Exoloratory Data Analysis

In [ ]:
category_count.plot(kind='bar',color='mediumorchid')
plt.xlabel('Categories')
plt.ylabel('Number of product sold')
plt.title('Product sold per category')
plt.show()

In [ ]:
item_and_season=df.groupby(['Item Purchased','Season']).size()
print(item_and_season)

In [ ]:
item_and_season=item_and_season.unstack()
print(item_and_season)

In [ ]:
item_and_season.isnull().sum()

In [ ]:
colors={'Fall':'Teal','Spring':'orange','Summer':'red','Winter':'cyan'}

In [ ]:
item_and_season.plot(kind='bar',stacked='true',colormap='viridis',color=[colors[season]for season in item_and_season.columns])
plt.xlabel("ITEM PURCHASED")
plt.ylabel("COUNT OF CUSTOMERS")
plt.title("count of Item sold per season")

In [ ]:
filter_season=input("enter the season you want to visualize ")
filter_condition=(df['Season']==filter_season)
filtered_df=df[filter_condition]

In [ ]:
item_and_season=filtered_df.groupby(['Item Purchased','Season']).size().unstack()
item_and_season.plot(kind='bar',stacked='true',colormap='viridis',color=[colors[season]for season in item_and_season.columns])
plt.xlabel("ITEM PURCHASED")
plt.ylabel("COUNT OF CUSTOMERS")
plt.title("count of Item sold per season")
plt.show()

# Recommendation System 

In [ ]:
from colorama import Fore, Style
from surprise import Dataset,Reader, SVD
from surprise.model_selection import train_test_split
reader = Reader(rating_scale=(1, 5))


unique_rating=sorted(df['Review Rating'].unique())
unique_rating

In [ ]:
surprise_data=Dataset.load_from_df(df[['Customer ID','Item Purchased','Review Rating']],reader)

trainset,testset=train_test_split(surprise_data,test_size=0.2,random_state=1)

model=SVD()

model.fit(trainset)

In [ ]:
while True :
    try:
        customer_identification_code=int(input('Enter your ID please : '))
        if customer_identification_code in df['Customer ID'].unique():
            break
        else :
            print(f"{Fore.YELLOW}INVALID ID{Style.RESET_ALL}")
    except ValueError:
        print(f"{Fore.RED}Invalid input. Please enter a valid integer.{Fore.RED}")

In [ ]:
products_user_rated=df[df['Customer ID']==customer_identification_code]['Item Purchased'].tolist()


all_products=df['Item Purchased'].unique()

unrated_products=set(all_products)-set(products_user_rated)

user_predictions = [model.predict(customer_identification_code, product) for product in unrated_products]


top_n=int(input('enter how many product recomendation you want '))
recommended_products=sorted(user_predictions,key=lambda x: x.est,reverse=True)[:top_n]
recommended_product_ids = [product.iid for product in recommended_products]

In [ ]:
print(f"\nTop{Fore.GREEN} {top_n} recommended products for Customer {customer_identification_code}: {recommended_product_ids}")